# Imports

In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

# Load Data

In [5]:
from src import utils
from datetime import datetime
import sqlite3

In [3]:
conn = utils.connect_athena(path='../configs/athena.yaml')

In [54]:
def load_cities(
    path=RAW_PATH / 'cities_metadata.csv'):

    return pd.read_csv(path)

def create_cities_partitions(config):
    
    cities = load_cities()
    
    c = cities.query(f'city_slug in {config["cities"]}')\
          .query(f'shape_type == "{config["shape_type"]}"')
    
    return c[['country_name', 
              'country_iso',
              'city_name',
              'city_slug',
              'timezone',
              'city_shapefile_wkt']].to_dict('records')

In [52]:
config = yaml.load("""
cities:
    - riodejaneiro
    - montevideo
    - saopaulo
    - lima
    - campinas
shape_type: city
""")

In [53]:
create_cities_partitions(config)

[{'country_name': 'Uruguay',
  'country_iso': 'UY',
  'city_name': 'Montevideo',
  'city_slug': 'montevideo',
  'timezone': 'America/Montevideo',
  'city_shapefile_wkt': 'POLYGON ((-56.4313997 -34.8281077, -56.4247744 -34.8377423, -56.3906016 -34.8452149, -56.3674538 -34.8652493, -56.3400567 -34.8707327, -56.3157268 -34.9048312, -56.2966594 -34.8931493, -56.256372 -34.9061487, -56.2582312 -34.8995178, -56.2447228 -34.8960775, -56.2467816 -34.8773478, -56.2121414 -34.875195, -56.1987604 -34.8840035, -56.2039271 -34.8885603, -56.1971708 -34.8967821, -56.2230036 -34.9059611, -56.2172778 -34.911533, -56.2229717 -34.9134515, -56.1715278 -34.9148245, -56.1604749 -34.938056, -56.145912 -34.913015, -56.1301366 -34.9112416, -56.1317517 -34.9054769, -56.1192069 -34.8986617, -56.0646307 -34.8996554, -56.0225006 -34.8777753, -56.0607703 -34.8658409, -56.0529689 -34.8162571, -56.0401115 -34.8049113, -56.0429654 -34.7922284, -56.0292274 -34.7809587, -56.0348759 -34.7689999, -56.0544133 -34.7605828, 

In [39]:
cities.to_csv(RAW_PATH / 'cities_metadata.csv', index=False)

,id,fua_id,country_name,country_iso,city_slug,city_name,population,timezone,shape_type,bbox,city_shapefile_geojson,city_shapefile_wkt,fua_shapefile_wkt
0,3986722,5327,México,MX,pueble,Municipio de Puebla,None,America/Mexico_City,city,"[-98.294121, 18.836779, -98.020233, 19.230938]","{'type': 'Polygon', 'coordinates': [[[-98.2941...","POLYGON ((-98.2941242 18.867063, -98.2802205 1...",MULTIPOLYGON (((-98.03334595111211 19.47370150...
1,7363416,337,Trinidad and Tobago,TT,sanfernado,San Fernando,56379,America/Port_of_Spain,city,"[-61.501808, 10.245889, -61.432804, 10.314652]","{'type': 'Polygon', 'coordinates': [[[-61.5018...","POLYGON ((-61.5018095 10.2463147, -61.4464073 ...",MULTIPOLYGON (((-61.39890264299142 10.39838433...
2,10331791,None,Belize,BZ,belpoman,Belmopan,13381,America/Belize,city,"[-88.799644, 17.21954, -88.730942, 17.276741]","{'type': 'Polygon', 'coordinates': [[[-88.7996...","POLYGON ((-88.79964150000001 17.2366656, -88.7...",None
3,10588599,6961,Brasil,BR,riodejaneiro,Região Metropolitana do Rio de Janeiro,None,America/Sao_Paulo,metro,"[-43.939568, -23.082705, -42.441795, -22.202368]","{'type': 'Polygon', 'coordinates': [[[-43.9395...","POLYGON ((-43.9395672 -22.87828, -43.9360428 -...",MULTIPOLYGON (((-43.12724669202157 -22.3292726...
4,3776660,17,Bahamas,BS,nassau,New Providence,None,America/Nassau,city,"[-77.7742, 24.790087, -76.832153, 25.559036]","{'type': 'Polygon', 'coordinates': [[[-77.7741...","POLYGON ((-77.77419740000001 25.1465684, -77.6...",MULTIPOLYGON (((-77.2875263134446 25.072865446...
5,332493,1477,Brasil,BR,manaus,Manaus,1598227,America/Manaus,city,"[-60.800999, -3.2219999, -59.16, -1.924]","{'type': 'Polygon', 'coordinates': [[[-60.801,...","POLYGON ((-60.801 -2.545, -60.756 -2.821, -60....",MULTIPOLYGON (((-60.02422546460881 -2.91204486...
6,303815,7390,Brasil,BR,maceio,Maceió,955010,America/Maceio,city,"[-35.816002, -9.7159948, -35.557999, -9.3699999]","{'type': 'Polygon', 'coordinates': [[[-35.816,...","POLYGON ((-35.816 -9.577, -35.769 -9.635, -35....",MULTIPOLYGON (((-35.82940382350298 -9.41412561...
7,5605470,3846,México,MX,aguascalientes,Municipio de Aguascalientes,None,America/Mexico_City,city,"[-102.58315, 21.622267, -102.06451, 22.075342]","{'type': 'Polygon', 'coordinates': [[[-102.583...","POLYGON ((-102.5831425 21.7471023, -102.552493...",MULTIPOLYGON (((-102.2599783850972 22.11270476...
8,4843685,6369,Brasil,BR,florianopolis,Microrregião de Florianópolis,None,America/Sao_Paulo,metro,"[-48.958, -28.094999, -48.327671, -27.268044]","{'type': 'Polygon', 'coordinates': [[[-48.958,...","POLYGON ((-48.958 -27.612, -48.951 -27.627, -4...",MULTIPOLYGON (((-48.56303259452466 -27.5011854...
9,1634207,906,Uruguay,UY,montevideo,Montevideo,1305865,America/Montevideo,city,"[-56.4314, -34.938057, -56.022499, -34.701851]","{'type': 'Polygon', 'coordinates': [[[-56.4313...","POLYGON ((-56.4313997 -34.8281077, -56.4247744...",MULTIPOLYGON (((-56.24260421895091 -34.6232432...


In [74]:
import requests

def get_holiday(country, year):
    url = 'https://calendarific.com/api/v2/holidays'

    params = dict(
        api_key='fc71997adad71795bb48409cdf1fb3f5ef2397e0',
        country=country,
        year=year
    )

    return requests.get(url, params).json()['response']

array(['MX', 'TT', 'BZ', 'BR', 'BS', 'UY', 'EC', 'BO', 'SR', 'NI', 'PE',
       'PA', 'CO', 'CL', 'BB', 'AR', 'HT', 'PY', 'CR', 'VE', 'DO', 'GT',
       'SV', 'JM'], dtype=object)

In [78]:
holidays = []
for country in cities['country_iso'].unique():
    for year in [2019, 2020]:
        try:
            holidays.append(get_holiday(country, year))
        except Exception as e:
            print(e)
            print(country, year)

In [82]:
import json
json.dump(holidays, open(RAW_PATH / 'holidays.json', 'w'))

In [84]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [85]:
a = flatten([h['holidays'] for h in holidays])

In [89]:
holidays = pd.io.json.json_normalize(a)

In [97]:
holidays = holidays[holidays['type'].apply(lambda x: 'Season' not in x)]

In [98]:
holidays.to_csv(RAW_PATH / 'holidays.csv', index=False)

In [99]:
holidays

,country.id,country.name,date.datetime.day,date.datetime.hour,date.datetime.minute,date.datetime.month,date.datetime.second,date.datetime.year,date.iso,date.timezone.offset,date.timezone.zoneabb,date.timezone.zonedst,date.timezone.zoneoffset,date.timezone.zonetotaloffset,description,locations,name,states,type
0,mx,Mexico,1,NaN,NaN,1,NaN,2019,2019-01-01,NaN,NaN,NaN,NaN,NaN,"New Year’s Day, on January 1, is the first day...",All,New Year's Day,All,[National holiday]
1,mx,Mexico,6,NaN,NaN,1,NaN,2019,2019-01-06,NaN,NaN,NaN,NaN,NaN,The Day of the Holy Kings is a Christian celeb...,All,Day of the Holy Kings,All,[Observance]
2,mx,Mexico,2,NaN,NaN,2,NaN,2019,2019-02-02,NaN,NaN,NaN,NaN,NaN,Candlemas is also known as the Feast of Presen...,All,Candlemas,All,[Observance]
3,mx,Mexico,4,NaN,NaN,2,NaN,2019,2019-02-04,NaN,NaN,NaN,NaN,NaN,Mexico’s Constitution Day is celebrated on the...,All,Day off for Constitution Day,All,[National holiday]
4,mx,Mexico,5,NaN,NaN,2,NaN,2019,2019-02-05,NaN,NaN,NaN,NaN,NaN,Mexico’s Constitution Day is celebrated on the...,All,Constitution Day,All,[National holiday]
5,mx,Mexico,14,NaN,NaN,2,NaN,2019,2019-02-14,NaN,NaN,NaN,NaN,NaN,Valentine’s Day in Mexico is not just for roma...,All,Valentine's Day,All,[Observance]
6,mx,Mexico,24,NaN,NaN,2,NaN,2019,2019-02-24,NaN,NaN,NaN,NaN,NaN,Mexico’s Flag Day celebrates the national flag...,All,Flag Day,All,[Observance]
7,mx,Mexico,6,NaN,NaN,3,NaN,2019,2019-03-06,NaN,NaN,NaN,NaN,NaN,"Catholics in Mexico observe Ash Wednesday, whi...",All,Ash Wednesday,All,"[Observance, Christian]"
8,mx,Mexico,18,NaN,NaN,3,NaN,2019,2019-03-18,NaN,NaN,NaN,NaN,NaN,Oil Expropriation Day serves as a national rem...,All,Oil Expropriation Day,All,[Observance]
9,mx,Mexico,18,NaN,NaN,3,NaN,2019,2019-03-18,NaN,NaN,NaN,NaN,NaN,Benito Juárez Day (Natalicio de Benito Juárez)...,All,Day off for Benito Juárez's Birthday Memorial,All,[National holiday]


In [148]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def load_sheets_tables():
    
    scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

    credentials = ServiceAccountCredentials.from_json_keyfile_name(
             '../configs/waze-corona.json', scope) # Your json file here

    gc = gspread.authorize(credentials)
    
    wks = gc.open('cities_metadata')

    datasets = {}
    for w in wks.worksheets():

        data = w.get_all_values()

        headers = data.pop(0)

        datasets[w.title] = pd.DataFrame(data, columns=headers)

    return datasets